In [ ]:
import mysql.connector
import pandas as pd

In [ ]:
class database:
    def __init__(self, host, user, password, database):
        self.host = host
        self.user = user
        self.password = password
        self.database = database
        self.conn = None
    def connect(self):
        self.conn = mysql.connector.connect(host=self.host,user=self.user,password=self.password)
        self.cursor = self.conn.cursor()
        self.cursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.database}")
        self.conn.database = self.database 
        print("success")
    def create_table(self,table_query):
        self.cursor.execute(table_query)
        self.conn.commit()
        print("success")
    def insert_data(self,insert_query, list_tuple):
        self.cursor.executemany(insert_query, list_tuple)
        self.conn.commit()
        print("success")
    def update(self, update_query):
        self.cursor.execute(update_query)
        self.conn.commit()
        print("Data updated successfully")
    def delete(self, delete_query):
        self.cursor.execute(delete_query)
        self.conn.commit()
        print("Data deleted successfully")
    def query(self, sql_query):
        self.cursor.execute(sql_query)
        results = self.cursor.fetchall()
        return results
    def close_connection(self):
        if self.cursor:
            self.cursor.close()
        if self.conn:
            self.conn.close()
        print("Connection closed successfully")

In [ ]:
config = {"host":"localhost", "user":"root","password":"root", "database":"zomato"}
db_obj = database(**config)

In [ ]:
db_obj.connect()

In [ ]:
customer_table = """CREATE TABLE customer_table (
    customer_id VARCHAR(255) PRIMARY KEY,
    name VARCHAR(255),
    email VARCHAR(255),
    phone VARCHAR(100),
    location VARCHAR(255),
    signup_date DATE,
    is_premium VARCHAR(10),
    preferred_cuisine VARCHAR(255),
    total_orders INT,
    average_rating FLOAT
)"""
db_obj.create_table(customer_table)

In [ ]:
customer_data = pd.read_csv(r"D:\DataScience_GUVI\0_Working_directory\01_Project_Zomato\cust_100.csv")

In [ ]:
customer_data_tuples = list(customer_data.itertuples(index=False, name=None))

In [ ]:
data = """INSERT INTO customer_table (
    customer_id,
    name,
    email,
    phone,
    location,
    signup_date,
    is_premium,
    preferred_cuisine,
    total_orders,
    average_rating
) VALUES (
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
);"""
db_obj.insert_data(data, customer_data_tuples)

In [ ]:
restaurants_table = """CREATE TABLE restaurants_table (
    restaurant_id VARCHAR(255) PRIMARY KEY,
    name VARCHAR(255),
    cuisine_type VARCHAR(50),
    location VARCHAR(255),
    owner_name VARCHAR(255),
    average_delivery_time FLOAT,
    contact_number VARCHAR(100),
    rating FLOAT,
    total_orders INT,
    is_active VARCHAR(10)
)"""
db_obj.create_table(restaurants_table)

In [ ]:
alter_table_query = """
ALTER TABLE restaurants_table
MODIFY average_delivery_time INT;
"""
db_obj.cursor.execute(alter_table_query)
db_obj.conn.commit()

In [ ]:
restaurants_table = pd.read_csv(r"D:\DataScience_GUVI\0_Working_directory\01_Project_Zomato\rest_100.csv")

In [ ]:
restaurants_data_tuples = list(customer_data.itertuples(index=False, name=None))

In [ ]:
for i, row in restaurants_table.iterrows():
    db_obj.cursor.execute("""
        INSERT INTO restaurants_table (
            restaurant_id, restaurant_name, cuisine_type, location, owner_name, 
            average_delivery_time, contact_number, rating, total_orders, is_active
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['restaurant_id'], 
        row['restaurant_name'], 
        row['cuisine_type'], 
        row['location'], 
        row['owner_name'], 
        int(row['average_delivery_time']), 
        int(row['contact_number']), 
        row['rating'], 
        row['total_orders'], 
        row['is_active']
    ))
db_obj.conn.commit()

In [ ]:
# data = """INSERT INTO restaurants_table (
#     restaurant_id,
#     restaurant_name,
#     cuisine_type,
#     location,
#     owner_name,
#     average_delivery_time,
#     contact_number,
#     rating,
#     total_orders,
#     is_active
# ) VALUES (
#     %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
# );"""
# db_obj.insert_data(data, restaurants_data_tuples)

In [ ]:
orders_table = """CREATE TABLE orders_table (
    order_id VARCHAR(255) PRIMARY KEY,
    customer_id VARCHAR(255),
    restaurant_id VARCHAR(255),
    order_date DATETIME,
    delivery_time DATETIME,
    status VARCHAR(10),
    total_amount DECIMAL(10, 2), 
    payment_mode VARCHAR(20), 
    discount_applied DECIMAL(5, 2),
    feedback_rating FLOAT,
    FOREIGN KEY (customer_id) REFERENCES customer_table(customer_id),
    FOREIGN KEY (restaurant_id) REFERENCES restaurants_table(restaurant_id)
)"""
db_obj.create_table(orders_table)

In [ ]:
orders_data = pd.read_csv(r"D:\DataScience_GUVI\0_Working_directory\01_Project_Zomato\order_100.csv")

In [ ]:
orders_data_tuples = list(orders_data.itertuples(index=False, name=None))

In [ ]:
data = """INSERT INTO orders_table (
    order_id,
    customer_id,
    restaurant_id,
    order_date,
    delivery_time,
    status,
    total_amount,
    payment_mode,
    discount_applied,
    feedback_rating
) VALUES (
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
);
"""
db_obj.insert_data(data, orders_data_tuples)

In [ ]:
delivery_persons_table = """CREATE TABLE delivery_persons_table (
    delivery_person_id VARCHAR(255) PRIMARY KEY,
    name VARCHAR(255),
    contact_number VARCHAR(100),
    vehicle_type VARCHAR(20),
    total_deliveries INT,
    average_rating FLOAT,
    location VARCHAR(255)
)"""
db_obj.create_table(delivery_persons_table)

In [ ]:
delivery_persons = pd.read_csv(r"D:\DataScience_GUVI\0_Working_directory\01_Project_Zomato\delivery_persons.csv")

In [ ]:
delivery_persons_data_tuples = list(delivery_persons.itertuples(index=False, name=None))

In [ ]:
data = """INSERT INTO delivery_persons_table (
    delivery_person_id,
    name,
    contact_number,
    vehicle_type,
    total_deliveries,
    average_rating,
    location
) VALUES (
    %s, %s, %s, %s, %s, %s, %s
);
"""
db_obj.insert_data(data, delivery_persons_data_tuples)

In [ ]:
deliveries_table = """CREATE TABLE deliveries_table (
    delivery_id VARCHAR(255) PRIMARY KEY,
    order_id VARCHAR(255),
    delivery_person_id VARCHAR(255),
    delivery_status VARCHAR(20),
    distance FLOAT,
    delivery_time INT,
    estimated_time INT,
    delivery_fee DECIMAL(5, 2),
    vehicle_type VARCHAR(20),
    FOREIGN KEY (order_id) REFERENCES orders_table(order_id),
    FOREIGN KEY (delivery_person_id) REFERENCES delivery_persons_table(delivery_person_id)
)"""
db_obj.create_table(deliveries_table)

In [ ]:
deliveries = pd.read_csv(r"D:\DataScience_GUVI\0_Working_directory\01_Project_Zomato\deliveries_100.csv")

In [ ]:
deliveries_data_tuples = list(deliveries.itertuples(index=False, name=None))

In [ ]:
data = """INSERT INTO deliveries_table (
    delivery_id,
    order_id,
    delivery_person_id,
    delivery_status,
    distance,
    delivery_time,
    estimated_time,
    delivery_fee,
    vehicle_type
) VALUES (
    %s, %s, %s, %s, %s, %s, %s, %s, %s
);
"""
db_obj.insert_data(data, deliveries_data_tuples)